In [33]:
import pandas as pd
import numpy as np
import os

from groq import Groq
from langchain_groq import ChatGroq
import chromadb
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.document_loaders import PyPDFLoader


In [2]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key:  ········


In [3]:

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
   
)

In [4]:
response = llm.invoke("In 2 sentences explain what is AI?")
response.content

'Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI involves the creation of algorithms and models that enable machines to process and analyze vast amounts of data, allowing them to make predictions, classify objects, and take actions autonomously.'

In [44]:
loader = WebBaseLoader("https://www.careerbeacon.com/en/job/2195743/scotiabank/senior-data-engineer/toronto")
loader.requests_per_second = 1
page_data = loader.load().pop().page_content
page_data

'\n\n\n\n\n\n\n\n\nSenior Data Engineer Job at Scotiabank | CareerBeacon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\tSearch Jobs\t\t\t\n\n\n\n\t\t\t\tSearch Employers\t\t\t\n\n\n\n\n\t\t\t\tSalary Tools\n\n\n\n\nSalary Guide\n\n\n\n\nIncome Tax Calculator\n\n\n\n\nSalary Converter\n\n\n\n\nMinimum Hourly Wage\n\n\n\n\nCost of Living Calculator\n\n\n\n\nExplore Demographics by Place\n\n\n\n\n\n\n\n\t\t\t\tResources\n\n\n\n\nFor students and new grads\n\n\n\n\nBlog\n\n\n\n\n\n\n\n\n\n\n\nFR\n\n\n\n\n\t\t\t\t\t\t\t\t\t\t\tPost a Job\t\t\t\t\t\t\t\t\t\n\n\n\nLog\xa0In\t\t\t\t\n\n\n\n\t\t\t\t\tSign Up\t\t\t\t\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch Jobs\n\n\n\n\n\n\nSearch Employers\n\n\n\n\n\n\nSalary Tools\n\n\n\n\n\n\nSalary Guide\n\n\n\n\nIncome Tax Calculator\n\n\n\n\nSalary Converter\n\n\n\n\nMinimum Hourly Wa

In [45]:
prompt_extract = PromptTemplate.from_template(
 """
 ##Job Description
 {page_data}
 ### Instruction:
 The scrapted text is from the career's page of a company's website. The instruction for you is to extract the job postings and return them in JSON format containing the 
 follwing keys:

Role:
Experience:
Skills (Give 5-10 skills in a list format):
Description:
Salary: (only if mentioned):

Only return the valid JSON file.

### Valid JSON Format without any preamble
 
 """ 
)

In [46]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input = {'page_data': page_data})
print(res.content)

{
  "Role": "Senior Data Engineer",
  "Experience": "10+ years of experience in data engineering, performance optimization for large OLTP applications",
  "Skills": [
    "Hadoop HDFS",
    "Hive",
    "Pig",
    "Flume",
    "Sqoop",
    "Google Cloud Platform Databases (SQL, Spanner, PostgreSQL)",
    "HQL",
    "Kafka",
    "Spark-streaming",
    "Infrastructure as Code (IaC) practices and frameworks like Terraform",
    "Java microservices and Spring Boot"
  ],
  "Description": "Leading development efforts in ingesting and transforming data from various sources. Working in the Big Data/ Hadoop environment, should be hands on in writing code, building scripts, writing specifications and responsible for end to end delivery of data in the Enterprise Data Lake environment",
  "Salary": null
}


In [47]:

json_parser = JsonOutputParser()
json_parsed_job_data = json_parser.parse(res.content)
json_parsed_job_data

{'Role': 'Senior Data Engineer',
 'Experience': '10+ years of experience in data engineering, performance optimization for large OLTP applications',
 'Skills': ['Hadoop HDFS',
  'Hive',
  'Pig',
  'Flume',
  'Sqoop',
  'Google Cloud Platform Databases (SQL, Spanner, PostgreSQL)',
  'HQL',
  'Kafka',
  'Spark-streaming',
  'Infrastructure as Code (IaC) practices and frameworks like Terraform',
  'Java microservices and Spring Boot'],
 'Description': 'Leading development efforts in ingesting and transforming data from various sources. Working in the Big Data/ Hadoop environment, should be hands on in writing code, building scripts, writing specifications and responsible for end to end delivery of data in the Enterprise Data Lake environment',
 'Salary': None}

## Extracting releavant skills from the PDF

In [48]:

file_path = "sample1.pdf"
loader = PyPDFLoader(file_path)
loader

In [49]:
loader = PyPDFLoader(
    "sample1.pdf",
    mode="single",
    pages_delimiter="\n-------THIS IS A CUSTOM END OF PAGE-------\n",
)
pdf_data  = loader.load()
print(docs[0].page_content[:5780])

incorrect startxref pointer(3)
parsing for Object Streams


Sarah Chen
 
Data Engineer | san.francisco@email.com | (415) 555
-
0123 | linkedin.com/in/sarachen | github.com/sarachen
 
PROFESSIONAL SUMMARY
 
Results
-
driven Data Engineer with 8+ years of experience designing and implementing scalable data pipelines, 
warehousing solutions, and ETL processes. Expertise in building robust data infrastructure across cloud 
platforms (AWS, Azure, GCP) supporting anal
ytics and ML applications. Proven track record of optimizing data 
systems that process petabytes of data daily.
 
TECHNICAL SKILLS
 
Languages: 
Python, SQL, Scala, Java, Bash
 
Data Processing: 
Apache Spark, Kafka, Airflow, Databricks, dbt, Flink
 
Cloud Platforms: 
AWS (Redshift, EMR, Glue, S3, Lambda), Azure (Synapse, Data Factory), GCP (BigQuery, 
Dataflow)
 
Databases: 
PostgreSQL, MySQL, MongoDB, Cassandra, Redis, Snowflake
 
Tools & Frameworks: 
Docker, Kubernetes, Terraform, Git, Jenkins, Tableau, Looker
 
PROFESSIONAL EXPERIENCE
 
Senior Data Engineer
 
| TechCorp Inc. | San 

In [50]:
prompt_extract_resume_pdf = PromptTemplate.from_template(
 """
 ##Job Description
 {pdf_data}
 ### Instruction:
 The scrapted text is from the pdf of the user. The instruction for you is to extract the skills, certifications and relavent experience and return them in JSON format containing the 
 follwing keys:

Skills:
Experience (3-4 lines based on the resume):
Certification:

Only return the valid JSON file.

### Valid JSON Format without any preamble
 
 """ 
)

In [54]:
chain_extract_pdf_data = prompt_extract_resume_pdf | llm
res_pdf = chain_extract_pdf_data.invoke(input = {'pdf_data': pdf_data})
print(res_pdf.content)

```json
{
  "Skills": {
    "Languages": ["Python", "SQL", "Scala", "Java", "Bash"],
    "Data Processing": ["Apache Spark", "Kafka", "Airflow", "Databricks", "dbt", "Flink"],
    "Cloud Platforms": ["AWS (Redshift, EMR, Glue, S3, Lambda)", "Azure (Synapse, Data Factory)", "GCP (BigQuery, Dataflow)"],
    "Databases": ["PostgreSQL", "MySQL", "MongoDB", "Cassandra", "Redis", "Snowflake"],
    "Tools & Frameworks": ["Docker", "Kubernetes", "Terraform", "Git", "Jenkins", "Tableau", "Looker"]
  },
  "Experience": [
    "Senior Data Engineer at TechCorp Inc. (June 2020 - Present)",
    "Architected and deployed real-time streaming data pipelines using Kafka and Spark Streaming, processing 5TB+ daily across 200+ microservices",
    "Reduced data warehouse query latency by 65% through optimization of Snowflake schemas and implementation of incremental materialized views",
    "Led migration of legacy ETL processes to modern ELT framework using dbt and Airflow, improving data freshness from 24

In [55]:

json_parser = JsonOutputParser()
json_parsed_pdf_data = json_parser.parse(res_pdf.content)
json_parsed_pdf_data

{'Skills': {'Languages': ['Python', 'SQL', 'Scala', 'Java', 'Bash'],
  'Data Processing': ['Apache Spark',
   'Kafka',
   'Airflow',
   'Databricks',
   'dbt',
   'Flink'],
  'Cloud Platforms': ['AWS (Redshift, EMR, Glue, S3, Lambda)',
   'Azure (Synapse, Data Factory)',
   'GCP (BigQuery, Dataflow)'],
  'Databases': ['PostgreSQL',
   'MySQL',
   'MongoDB',
   'Cassandra',
   'Redis',
   'Snowflake'],
  'Tools & Frameworks': ['Docker',
   'Kubernetes',
   'Terraform',
   'Git',
   'Jenkins',
   'Tableau',
   'Looker']},
 'Experience': ['Senior Data Engineer at TechCorp Inc. (June 2020 - Present)',
  'Architected and deployed real-time streaming data pipelines using Kafka and Spark Streaming, processing 5TB+ daily across 200+ microservices',
  'Reduced data warehouse query latency by 65% through optimization of Snowflake schemas and implementation of incremental materialized views',
  'Led migration of legacy ETL processes to modern ELT framework using dbt and Airflow, improving data fr

In [65]:
prompt_comparsion = PromptTemplate.from_template(
 """
 ##Job Description
 {json_parsed_job_data}, 
 
 ###User resume data 
 {json_parsed_pdf_data}
 ### Instruction:

 
 You are a smart job matcher. You already have two jsons. The jsons are returned from job description and the pdf (resume) which the user uploads.
 Instructions for you
 1. Compare both the job skills vs pdf data json and return the matching skills. Use semantic searching matching
 2. Return the list of skills that the user lacks based on the job data
 3. Give a score out of 10 based on the job suitability
 4. Give a one-two liner whether you advise the person to apply for the job or not purely based on the skills provided

### Return a response in the following JSON format without preamble:

Matched Skills: 
Unmatched Skills:
Job Suitability Score
Advice:


 """ 
    
)

In [67]:
chain_extract_jop_matching = prompt_comparsion | llm
res_matching = chain_extract_jop_matching.invoke(input = {'json_parsed_job_data': json_parsed_job_data, "json_parsed_pdf_data":json_parsed_pdf_data})
print(res_matching.content)


json_parsed_final_output = json_parser.parse(res_matching.content)
json_parsed_final_output



{
  "Matched Skills": [
    "Hadoop HDFS",
    "Hive",
    "Pig",
    "Flume",
    "Sqoop",
    "Kafka",
    "Spark-streaming",
    "Infrastructure as Code (IaC) practices and frameworks like Terraform",
    "Java microservices and Spring Boot",
    "PostgreSQL"
  ],
  "Unmatched Skills": [
    "Google Cloud Platform Databases (SQL, Spanner)",
    "HQL",
    "Cloud Platforms",
    "Databases",
    "Tools & Frameworks"
  ],
  "Job Suitability Score": 8,
  "Advice": "Based on the skills provided, it is advised to apply for the job as the candidate has a strong foundation in data engineering and related technologies, but may require additional learning to fill the gaps in Google Cloud Platform and HQL."
}


{'Matched Skills': ['Hadoop HDFS',
  'Hive',
  'Pig',
  'Flume',
  'Sqoop',
  'Kafka',
  'Spark-streaming',
  'Infrastructure as Code (IaC) practices and frameworks like Terraform',
  'Java microservices and Spring Boot',
  'PostgreSQL'],
 'Unmatched Skills': ['Google Cloud Platform Databases (SQL, Spanner)',
  'HQL',
  'Cloud Platforms',
  'Databases',
  'Tools & Frameworks'],
 'Job Suitability Score': 8,
 'Advice': 'Based on the skills provided, it is advised to apply for the job as the candidate has a strong foundation in data engineering and related technologies, but may require additional learning to fill the gaps in Google Cloud Platform and HQL.'}